In [1]:
!pip install sexpdata

In [20]:
import subprocess, sexpdata
from sexpdata import loads

proc = subprocess.Popen(
    ["sertop"],
    stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)

def read(_proc):
    line = "("
    while "Completed" not in line:
        line += _proc.stdout.readline()
        print(line.split("\n")[-2])
    return line + ")"

line = proc.stdout.readline()
while "(contents Processed)" not in line:
    line = proc.stdout.readline()


def add(_proc, line):
    cmd = f'(Add () "{line}")\n'
    proc.stdin.write(cmd)
    proc.stdin.flush()
    return read(proc)

In [21]:
cmd = '(Add () "Require Import Coq.Lists.List.")\n'
proc.stdin.write(cmd)
proc.stdin.flush()
read(proc)


((Answer 0 Ack)
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Lists.List /root/.opam/4.14.1/lib/coq/theories/Lists/List.vo))))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Arith.PeanoNat /root/.opam/4.14.1/lib/coq/theories/Arith/PeanoNat.vo))))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Numbers.Natural.Abstract.NAxioms /root/.opam/4.14.1/lib/coq/theories/Numbers/Natural/Abstract/NAxioms.vo))))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Bool.Bool /root/.opam/4.14.1/lib/coq/theories/Bool/Bool.vo))))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Classes.DecidableClass /root/.opam/4.14.1/lib/coq/theories/Classes/DecidableClass.vo))))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Numbers.NatInt.NZAxioms /root/.opam/4.14.1/lib/coq/theorie

'((Answer 0 Ack)\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents(ProcessingIn master))))\n(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Lists.List /root/.opam/4.14.1/lib/coq/theories/Lists/List.vo))))\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Arith.PeanoNat /root/.opam/4.14.1/lib/coq/theories/Arith/PeanoNat.vo))))\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Numbers.Natural.Abstract.NAxioms /root/.opam/4.14.1/lib/coq/theories/Numbers/Natural/Abstract/NAxioms.vo))))\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Bool.Bool /root/.opam/4.14.1/lib/coq/theories/Bool/Bool.vo))))\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Classes.DecidableClass /root/.opam/4.14.1/lib/coq/theories/Classes/DecidableClass.vo))))\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents(FileLoaded Coq.Numbers.NatInt.NZAxioms /root/.opam/4.14.1/lib/co

In [22]:
cmd = '(Add () "Import ListNotations.")\n'
proc.stdin.write(cmd)
proc.stdin.flush()
read(proc)

((Answer 1 Ack)
(Feedback((doc_id 0)(span_id 3)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))
(Answer 1(Added 3((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 21))NewAddTip))
(Answer 1 Completed)


'((Answer 1 Ack)\n(Feedback((doc_id 0)(span_id 3)(route 0)(contents(ProcessingIn master))))\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))\n(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))\n(Answer 1(Added 3((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 21))NewAddTip))\n(Answer 1 Completed)\n)'

In [23]:
#cmd = '(Add () "Lemma test: ~ (exists (l : list nat), forall x, In x l).")\n'
#cmd = '(Add () "Lemma test: forall (x y: nat), x = y -> forall (z : nat), y = z -> x = z.")\n'
cmd = '(Add () "Lemma test: forall (x : list nat), x = x.")\n'
proc.stdin.write(cmd)
proc.stdin.flush()

In [24]:
read(proc)

((Answer 2 Ack)
(Answer 2(Added 4((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 41))NewAddTip))
(Answer 2 Completed)


'((Answer 2 Ack)\n(Answer 2(Added 4((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 41))NewAddTip))\n(Answer 2 Completed)\n)'

In [25]:
# Выполняем добавленный фрагмент с id = 2
cmd = '(Exec 4)\n'
proc.stdin.write(cmd)
proc.stdin.flush()

# Читаем ответы
read(proc)


((Answer 3 Ack)
(Feedback((doc_id 0)(span_id 4)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 4)(route 0)(contents Processed)))
(Answer 3 Completed)


'((Answer 3 Ack)\n(Feedback((doc_id 0)(span_id 4)(route 0)(contents(ProcessingIn master))))\n(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))\n(Feedback((doc_id 0)(span_id 4)(route 0)(contents Processed)))\n(Answer 3 Completed)\n)'

In [26]:
cmd = '(Add () "intros x y.")\n'
proc.stdin.write(cmd)
proc.stdin.flush()

# Читаем ответы
read(proc)

cmd = '(Exec 5)\n'
proc.stdin.write(cmd)
proc.stdin.flush()

# Читаем ответы
all_ans = loads(read(proc))

((Answer 4 Ack)
(Answer 4(Added 5((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 11))NewAddTip))
(Answer 4 Completed)
((Answer 5 Ack)
(Feedback((doc_id 0)(span_id 5)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 4)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 5)(route 0)(contents(Message(level Error)(loc(((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 10))))(pp(Pp_string"No product even after head-reduction."))(str"No product even after head-reduction.")))))
(Answer 5(CoqExn((loc(((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 10))))(stm_ids((4 5)))(backtrace(Backtrace()))(exn("Logic_monad.TacticFailure(_)"))(pp(Pp_string"No product even after head-reduction."))(str"No product even after head-reduction."))))
(Answer 5 Completed)


In [14]:
cmd = '(Query ((sid 6)) Goals)\n'
proc.stdin.write(cmd)
proc.stdin.flush()

# Читаем ответы
all_ans = loads(read(proc))

((Answer 15 Ack)
(Answer 15(ObjList((CoqGoal((goals(((info((evar(Ser_Evar 2))(name())))(ty(Prod((binder_name(Name(Id x)))(binder_relevance Relevant))(App(Ind(((MutInd(KerName(MPfile(DirPath((Id Datatypes)(Id Init)(Id Coq))))(Id list))())0)(Instance(()()))))((Ind(((MutInd(KerName(MPfile(DirPath((Id Datatypes)(Id Init)(Id Coq))))(Id nat))())0)(Instance(()()))))))(App(Ind(((MutInd(KerName(MPfile(DirPath((Id Logic)(Id Init)(Id Coq))))(Id eq))())0)(Instance(()()))))((App(Ind(((MutInd(KerName(MPfile(DirPath((Id Datatypes)(Id Init)(Id Coq))))(Id list))())0)(Instance(()()))))((Ind(((MutInd(KerName(MPfile(DirPath((Id Datatypes)(Id Init)(Id Coq))))(Id nat))())0)(Instance(()()))))))(Rel 1)(Rel 1)))))(hyp()))))(stack())(bullet())(shelf())(given_up()))))))
(Answer 15 Completed)


In [19]:
all_ans

[[Symbol('Answer'), 19, Symbol('Ack')],
 [Symbol('Feedback'),
  [[Symbol('doc_id'), 0],
   [Symbol('span_id'), 7],
   [Symbol('route'), 0],
   [Symbol('contents'), Symbol('Processed')]]],
 [Symbol('Answer'),
  19,
  [Symbol('CoqExn'),
   [[Symbol('loc'),
     [[[Symbol('fname'), Symbol('ToplevelInput')],
       [Symbol('line_nb'), 1],
       [Symbol('bol_pos'), 0],
       [Symbol('line_nb_last'), 1],
       [Symbol('bol_pos_last'), 0],
       [Symbol('bp'), 0],
       [Symbol('ep'), 10]]]],
    [Symbol('stm_ids'), [[6, 7]]],
    [Symbol('backtrace'), [Symbol('Backtrace'), []]],
    [Symbol('exn'), ['Logic_monad.TacticFailure(_)']],
    [Symbol('pp'),
     [Symbol('Pp_string'), 'No product even after head-reduction.']],
    [Symbol('str'), 'No product even after head-reduction.']]]],
 [Symbol('Answer'), 19, Symbol('Completed')]]

In [15]:
def find_a_leters(tree, leters, path="0"):
    results = []
    if isinstance(tree, list) or isinstance(tree, tuple):
        if tree and isinstance(tree[0], sexpdata.Symbol) and tree[0].value() in leters:
            cur_tree = []
            for i, elem in enumerate(tree):
                if i != 0 and elem and isinstance(elem, list) \
                    and isinstance(elem[0], sexpdata.Symbol) and elem[0].value() in leters:
                    break
                cur_tree.append(elem)
            results.append((path, cur_tree))
        for i, elem in enumerate(tree):
            results += find_a_leters(elem, leters, path + "." + str(i))
    return results

def extract_name(name_tree):
    # ищем (Name (Id x))
    for elem in name_tree:
        if isinstance(elem, list) and elem and isinstance(elem[0], sexpdata.Symbol):
            if elem[0].value() == "Id":
                return elem[1].value()
    return None

def extract_ind(ind_tree):
    # ищем (MutInd (... (Id nat) ...))
    return ind_tree
    for elem in ind_tree:
        if isinstance(elem, list):
            for sub in elem:
                if isinstance(sub, list) and sub and isinstance(sub[0], sexpdata.Symbol):
                    if sub[0].value() == "Id":
                        return sub[1].value()
    return None


def find_forall(tree):
    prods = find_a_leters(tree, ["Prod", "App"])
    results = []
    for path, prod_tree in prods:
        names = find_a_leters(prod_tree, ["Name"], path)
        tipes = find_a_leters(prod_tree, ["Ind"], path)
        if names and tipes:
            var = extract_name(names[0][1])
            tipe = extract_ind(tipes[0][1])
            if var and tipe:
                results.append(("forall", var, tipe))
    return results
        

In [16]:
find_forall(all_ans)

[]

In [43]:
for x in find_a_leters(all_ans, ["Prod"]):
    print (x[0])

0.1.2.1.0.1.0.1.0.1.1
0.1.2.1.0.1.0.1.0.1.1.3
0.1.2.1.0.1.0.1.0.1.1.3.3
0.1.2.1.0.1.0.1.0.1.1.3.3.3
0.1.2.1.0.1.0.1.0.1.1.3.3.3.3
